# Collaborative Filtering Evaluation Using Cosine Similarity

This notebook evaluates the collaborative filtering algorithm in `collaborative_filtering.py` using only cosine similarity-based methods. We'll test accuracy, performance, and recommendation quality.

## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time
import warnings
warnings.filterwarnings('ignore')

# Import our collaborative filtering module
import sys
sys.path.append('.')
from algorithm.collaborative_filtering import CollaborativeFilter, get_recommendations

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All imports successful!")

## 2. Load and Prepare Data

In [ ]:
# Load movie dataset
print("📥 Loading movie dataset...")
movies_df = pd.read_csv('dataset/movies.csv')

# Display basic dataset info
print(f"Dataset shape: {movies_df.shape}")
print(f"Columns: {list(movies_df.columns)}")
print("\nFirst 5 movies:")
movies_df.head()

In [ ]:
# Load user ratings data
print("👥 Loading user ratings data...")
import json

try:
    with open('users_data.json', 'r') as f:
        users_data = json.load(f)
    
    # Extract ratings from users data
    all_user_ratings = {}
    if 'users' in users_data:
        for user_id, user_info in users_data['users'].items():
            if 'ratings' in user_info and user_info['ratings']:
                all_user_ratings[user_id] = user_info['ratings']
    
    print(f"Total users with ratings: {len(all_user_ratings)}")
    if all_user_ratings:
        total_ratings = sum(len(ratings) for ratings in all_user_ratings.values())
        print(f"Total ratings: {total_ratings}")
        avg_ratings_per_user = total_ratings / len(all_user_ratings)
        print(f"Average ratings per user: {avg_ratings_per_user:.1f}")
    else:
        print("⚠️ No user ratings found, creating synthetic data for testing...")
        all_user_ratings = create_synthetic_ratings(movies_df)
        
except FileNotFoundError:
    print("⚠️ users_data.json not found, creating synthetic data for testing...")
    all_user_ratings = create_synthetic_ratings(movies_df)

def create_synthetic_ratings(movies_df, n_users=50, n_ratings_per_user=20):
    """Create synthetic user ratings for testing purposes"""
    synthetic_ratings = {}
    movie_ids = movies_df['id'].tolist()[:1000]  # Use first 1000 movies
    
    for user_id in range(n_users):
        # Randomly select movies for this user
        user_movies = np.random.choice(movie_ids, n_ratings_per_user, replace=False)
        user_ratings = {}
        
        for movie_id in user_movies:
            # Generate ratings with some bias towards higher ratings
            rating = np.random.choice([5,6,7,8,9,10], p=[0.05,0.1,0.2,0.3,0.25,0.1])
            user_ratings[str(movie_id)] = rating
            
        synthetic_ratings[f"user_{user_id}"] = user_ratings
    
    return synthetic_ratings

# If we created synthetic data, use it
if not all_user_ratings:
    all_user_ratings = create_synthetic_ratings(movies_df)
    print(f"✅ Created synthetic data: {len(all_user_ratings)} users")

print("\nSample user ratings:")
sample_user = list(all_user_ratings.keys())[0]
print(f"User '{sample_user}' has {len(all_user_ratings[sample_user])} ratings")
print(f"Sample ratings: {dict(list(all_user_ratings[sample_user].items())[:5])}")

## 3. Cosine Similarity Implementation & Testing

In [ ]:
class CosineSimilarityEvaluator:
    """Pure cosine similarity-based collaborative filtering evaluator"""
    
    def __init__(self, movies_df, user_ratings_dict):
        self.movies_df = movies_df
        self.user_ratings_dict = user_ratings_dict
        self.user_item_matrix = None
        self.similarity_matrix = None
        self._build_user_item_matrix()
        
    def _build_user_item_matrix(self):
        """Build user-item rating matrix"""
        print("🔨 Building user-item matrix...")
        
        # Get all unique movie IDs from ratings
        all_movie_ids = set()
        for user_ratings in self.user_ratings_dict.values():
            all_movie_ids.update(user_ratings.keys())
        
        all_movie_ids = sorted(list(all_movie_ids))
        user_ids = list(self.user_ratings_dict.keys())
        
        print(f"Matrix dimensions: {len(user_ids)} users × {len(all_movie_ids)} movies")
        
        # Create matrix
        matrix_data = np.zeros((len(user_ids), len(all_movie_ids)))
        
        for i, user_id in enumerate(user_ids):
            user_ratings = self.user_ratings_dict[user_id]
            for j, movie_id in enumerate(all_movie_ids):
                if movie_id in user_ratings:
                    matrix_data[i, j] = user_ratings[movie_id]
        
        self.user_item_matrix = pd.DataFrame(
            matrix_data, 
            index=user_ids, 
            columns=all_movie_ids
        )
        
        print(f"✅ Matrix built successfully!")
        print(f"Sparsity: {(matrix_data == 0).sum() / matrix_data.size * 100:.1f}%")
        
    def calculate_user_similarity(self, target_user_id, method='cosine'):
        """Calculate similarity between target user and all other users"""
        if target_user_id not in self.user_item_matrix.index:
            return pd.Series(dtype=float)
            
        target_ratings = self.user_item_matrix.loc[target_user_id].values.reshape(1, -1)
        all_ratings = self.user_item_matrix.values
        
        # Calculate cosine similarity
        similarities = cosine_similarity(target_ratings, all_ratings)[0]
        
        return pd.Series(similarities, index=self.user_item_matrix.index)
    
    def predict_rating(self, user_id, movie_id, k=10):
        """Predict rating using k most similar users"""
        if user_id not in self.user_item_matrix.index:
            return self.movies_df[self.movies_df['id'].astype(str) == movie_id]['vote_average'].iloc[0] if len(self.movies_df[self.movies_df['id'].astype(str) == movie_id]) > 0 else 5.0
        
        # Get user similarities
        similarities = self.calculate_user_similarity(user_id)
        
        # Remove target user and get top k similar users
        similarities = similarities.drop(user_id, errors='ignore')
        top_similar = similarities.nlargest(k)
        
        # Get ratings for the movie from similar users
        weighted_sum = 0
        similarity_sum = 0
        
        for similar_user, similarity_score in top_similar.items():
            if similarity_score > 0.1:  # Minimum similarity threshold
                if movie_id in self.user_ratings_dict[similar_user]:
                    rating = self.user_ratings_dict[similar_user][movie_id]
                    weighted_sum += similarity_score * rating
                    similarity_sum += similarity_score
        
        if similarity_sum > 0:
            predicted_rating = weighted_sum / similarity_sum
            return max(1, min(10, predicted_rating))
        else:
            # Fallback to movie's average rating
            movie_avg = self.movies_df[self.movies_df['id'].astype(str) == movie_id]['vote_average']
            return movie_avg.iloc[0] if len(movie_avg) > 0 else 5.0
    
    def recommend_movies(self, user_id, n_recommendations=10, k=10):
        """Recommend movies using cosine similarity"""
        if user_id not in self.user_ratings_dict:
            return pd.DataFrame()
        
        user_ratings = self.user_ratings_dict[user_id]
        rated_movies = set(user_ratings.keys())
        
        # Get all movies not rated by the user
        all_movies = set(self.movies_df['id'].astype(str))
        unrated_movies = all_movies - rated_movies
        
        # Predict ratings for unrated movies
        predictions = []
        for movie_id in list(unrated_movies)[:500]:  # Limit for performance
            predicted_rating = self.predict_rating(user_id, movie_id, k)
            predictions.append((movie_id, predicted_rating))
        
        # Sort by predicted rating
        predictions.sort(key=lambda x: x[1], reverse=True)
        
        # Get movie details for top recommendations
        recommended_movies = []
        for movie_id, pred_rating in predictions[:n_recommendations]:
            movie_info = self.movies_df[self.movies_df['id'].astype(str) == movie_id]
            if not movie_info.empty:
                movie_dict = movie_info.iloc[0].to_dict()
                movie_dict['predicted_rating'] = pred_rating
                recommended_movies.append(movie_dict)
        
        return pd.DataFrame(recommended_movies)

# Initialize evaluator
print("🚀 Initializing Cosine Similarity Evaluator...")
evaluator = CosineSimilarityEvaluator(movies_df, all_user_ratings)
print("✅ Evaluator ready!")

## 4. Evaluation Metrics

In [ ]:
def evaluate_prediction_accuracy(evaluator, test_size=0.2, k_values=[5, 10, 20]):
    """Evaluate prediction accuracy using train-test split"""
    print("🎯 Evaluating Prediction Accuracy...")
    
    results = []
    
    # Select users with enough ratings for meaningful evaluation
    suitable_users = [
        user_id for user_id, ratings in all_user_ratings.items() 
        if len(ratings) >= 10
    ][:20]  # Limit to 20 users for performance
    
    print(f"Testing with {len(suitable_users)} users...")
    
    for k in k_values:
        print(f"\n📊 Testing with k={k} similar users...")
        
        mae_scores = []
        rmse_scores = []
        
        for i, user_id in enumerate(suitable_users):
            if i % 5 == 0:
                print(f"  Processing user {i+1}/{len(suitable_users)}...")
            
            user_ratings = all_user_ratings[user_id]
            
            # Split ratings into train and test
            items = list(user_ratings.items())
            train_items, test_items = train_test_split(items, test_size=test_size, random_state=42)
            
            train_ratings = dict(train_items)
            test_ratings = dict(test_items)
            
            # Create temporary evaluator with train data only
            temp_user_ratings = all_user_ratings.copy()
            temp_user_ratings[user_id] = train_ratings
            temp_evaluator = CosineSimilarityEvaluator(movies_df, temp_user_ratings)
            
            # Predict ratings for test movies
            predictions = []
            actuals = []
            
            for movie_id, actual_rating in test_items:
                predicted = temp_evaluator.predict_rating(user_id, movie_id, k)
                predictions.append(predicted)
                actuals.append(actual_rating)
            
            if predictions and actuals:
                mae = mean_absolute_error(actuals, predictions)
                rmse = np.sqrt(mean_squared_error(actuals, predictions))
                mae_scores.append(mae)
                rmse_scores.append(rmse)
        
        avg_mae = np.mean(mae_scores) if mae_scores else float('inf')
        avg_rmse = np.mean(rmse_scores) if rmse_scores else float('inf')
        
        results.append({
            'k': k,
            'MAE': avg_mae,
            'RMSE': avg_rmse,
            'num_users_tested': len(mae_scores)
        })
        
        print(f"  MAE: {avg_mae:.3f}, RMSE: {avg_rmse:.3f}")
    
    return pd.DataFrame(results)

# Run prediction accuracy evaluation
accuracy_results = evaluate_prediction_accuracy(evaluator)
print("\n📈 Prediction Accuracy Results:")
accuracy_results

## 5. Recommendation Quality Analysis

In [ ]:
def analyze_recommendation_quality(evaluator, sample_users=5):
    """Analyze quality and diversity of recommendations"""
    print("🎬 Analyzing Recommendation Quality...")
    
    sample_user_ids = list(all_user_ratings.keys())[:sample_users]
    
    quality_metrics = []
    
    for user_id in sample_user_ids:
        print(f"\n👤 User: {user_id}")
        
        # Get user's actual preferences
        user_ratings = all_user_ratings[user_id]
        avg_user_rating = np.mean(list(user_ratings.values()))
        
        print(f"  User's average rating: {avg_user_rating:.1f}")
        print(f"  Number of rated movies: {len(user_ratings)}")
        
        # Get top rated movies by user
        top_rated = sorted(user_ratings.items(), key=lambda x: x[1], reverse=True)[:3]
        print("  Top rated movies:")
        for movie_id, rating in top_rated:
            movie_title = movies_df[movies_df['id'].astype(str) == movie_id]['title']
            title = movie_title.iloc[0] if len(movie_title) > 0 else f"Movie {movie_id}"
            print(f"    - {title}: {rating}/10")
        
        # Get recommendations
        recommendations = evaluator.recommend_movies(user_id, n_recommendations=10)
        
        if not recommendations.empty:
            print("  \n🎯 Top 5 Recommendations:")
            for i, (_, movie) in enumerate(recommendations.head(5).iterrows()):
                print(f"    {i+1}. {movie['title']} (Predicted: {movie['predicted_rating']:.1f}, Actual: {movie['vote_average']:.1f})")
            
            # Calculate quality metrics
            avg_predicted_rating = recommendations['predicted_rating'].mean()
            avg_actual_rating = recommendations['vote_average'].mean()
            
            # Genre diversity
            all_genres = set()
            for genres_str in recommendations['genre'].fillna(''):
                genres = [g.strip() for g in str(genres_str).split(',')]
                all_genres.update(genres)
            genre_diversity = len(all_genres)
            
            # Popularity distribution
            popularity_std = recommendations['popularity'].std()
            
            quality_metrics.append({
                'user_id': user_id,
                'user_avg_rating': avg_user_rating,
                'num_user_ratings': len(user_ratings),
                'avg_predicted_rating': avg_predicted_rating,
                'avg_actual_rating': avg_actual_rating,
                'genre_diversity': genre_diversity,
                'popularity_diversity': popularity_std
            })
            
            print(f"  📊 Quality Metrics:")
            print(f"    Average predicted rating: {avg_predicted_rating:.2f}")
            print(f"    Average actual rating: {avg_actual_rating:.2f}")
            print(f"    Genre diversity: {genre_diversity} unique genres")
        else:
            print("  ⚠️ No recommendations generated")
    
    return pd.DataFrame(quality_metrics)

# Run recommendation quality analysis
quality_results = analyze_recommendation_quality(evaluator)
print("\n📊 Quality Analysis Summary:")
quality_results

## 6. Performance Benchmarking

In [ ]:
def benchmark_performance(evaluator):
    """Benchmark performance of different components"""
    print("⚡ Benchmarking Performance...")
    
    sample_users = list(all_user_ratings.keys())[:10]
    performance_results = []
    
    # Test similarity calculation
    print("\n🔍 Testing similarity calculation...")
    start_time = time.time()
    
    for user_id in sample_users:
        similarities = evaluator.calculate_user_similarity(user_id)
    
    similarity_time = time.time() - start_time
    print(f"  Time for {len(sample_users)} similarity calculations: {similarity_time:.3f}s")
    print(f"  Average time per similarity calculation: {similarity_time/len(sample_users):.3f}s")
    
    # Test rating prediction
    print("\n🎯 Testing rating prediction...")
    start_time = time.time()
    
    test_movies = list(movies_df['id'].astype(str))[:50]  # First 50 movies
    predictions_made = 0
    
    for user_id in sample_users[:5]:  # Test with 5 users
        for movie_id in test_movies[:10]:  # Test with 10 movies each
            evaluator.predict_rating(user_id, movie_id)
            predictions_made += 1
    
    prediction_time = time.time() - start_time
    print(f"  Time for {predictions_made} predictions: {prediction_time:.3f}s")
    print(f"  Average time per prediction: {prediction_time/predictions_made:.4f}s")
    
    # Test full recommendation generation
    print("\n🎬 Testing recommendation generation...")
    start_time = time.time()
    
    for user_id in sample_users[:3]:  # Test with 3 users
        recommendations = evaluator.recommend_movies(user_id, n_recommendations=10)
    
    recommendation_time = time.time() - start_time
    print(f"  Time for 3 full recommendation sets: {recommendation_time:.3f}s")
    print(f"  Average time per recommendation set: {recommendation_time/3:.3f}s")
    
    # Memory usage analysis
    print("\n💾 Memory Usage Analysis:")
    matrix_size = evaluator.user_item_matrix.memory_usage(deep=True).sum() / 1024**2
    print(f"  User-item matrix size: {matrix_size:.1f} MB")
    print(f"  Matrix dimensions: {evaluator.user_item_matrix.shape}")
    
    return {
        'similarity_time_per_user': similarity_time/len(sample_users),
        'prediction_time_per_rating': prediction_time/predictions_made,
        'recommendation_time_per_user': recommendation_time/3,
        'matrix_memory_mb': matrix_size
    }

# Run performance benchmark
perf_results = benchmark_performance(evaluator)
print("\n⚡ Performance Summary:")
for metric, value in perf_results.items():
    print(f"  {metric}: {value}")

## 7. Comparison with Original Algorithm

In [ ]:
def compare_with_original_algorithm():
    """Compare pure cosine similarity with original hybrid algorithm"""
    print("🔄 Comparing with Original Algorithm...")
    
    # Test with sample users
    test_users = list(all_user_ratings.keys())[:3]
    comparison_results = []
    
    for user_id in test_users:
        user_ratings = all_user_ratings[user_id]
        
        print(f"\n👤 Testing User: {user_id}")
        print(f"  User has {len(user_ratings)} ratings")
        
        # Get recommendations from pure cosine similarity
        start_time = time.time()
        cosine_recs = evaluator.recommend_movies(user_id, n_recommendations=10)
        cosine_time = time.time() - start_time
        
        # Get recommendations from original algorithm
        start_time = time.time()
        original_recs = get_recommendations(movies_df, user_ratings, n_recommendations=10)
        original_time = time.time() - start_time
        
        print(f"  🕐 Cosine Similarity Time: {cosine_time:.3f}s")
        print(f"  🕑 Original Algorithm Time: {original_time:.3f}s")
        
        # Compare recommendations
        cosine_titles = set(cosine_recs['title'].tolist() if not cosine_recs.empty else [])
        original_titles = set(original_recs['title'].tolist() if not original_recs.empty else [])
        
        overlap = len(cosine_titles.intersection(original_titles))
        
        print(f"  📊 Recommendation Overlap: {overlap}/10 movies")
        
        if not cosine_recs.empty and not original_recs.empty:
            avg_cosine_rating = cosine_recs['vote_average'].mean()
            avg_original_rating = original_recs['vote_average'].mean()
            
            print(f"  ⭐ Average Rating (Cosine): {avg_cosine_rating:.2f}")
            print(f"  ⭐ Average Rating (Original): {avg_original_rating:.2f}")
            
            comparison_results.append({
                'user_id': user_id,
                'cosine_time': cosine_time,
                'original_time': original_time,
                'overlap': overlap,
                'cosine_avg_rating': avg_cosine_rating,
                'original_avg_rating': avg_original_rating
            })
        
        # Show sample recommendations
        print("  🎯 Top 3 Cosine Similarity Recommendations:")
        if not cosine_recs.empty:
            for i, (_, movie) in enumerate(cosine_recs.head(3).iterrows()):
                print(f"    {i+1}. {movie['title']} ({movie['vote_average']:.1f})")
        else:
            print("    None")
            
        print("  🎯 Top 3 Original Algorithm Recommendations:")
        if not original_recs.empty:
            for i, (_, movie) in enumerate(original_recs.head(3).iterrows()):
                print(f"    {i+1}. {movie['title']} ({movie['vote_average']:.1f})")
        else:
            print("    None")
    
    return pd.DataFrame(comparison_results)

# Run comparison
comparison_results = compare_with_original_algorithm()

if not comparison_results.empty:
    print("\n📈 Overall Comparison Summary:")
    print(f"Average Cosine Similarity Time: {comparison_results['cosine_time'].mean():.3f}s")
    print(f"Average Original Algorithm Time: {comparison_results['original_time'].mean():.3f}s")
    print(f"Average Recommendation Overlap: {comparison_results['overlap'].mean():.1f}/10")
    
    comparison_results

## 8. Visualization & Results

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Collaborative Filtering Evaluation Results', fontsize=16)

# 1. Prediction Accuracy vs K
if not accuracy_results.empty:
    axes[0,0].plot(accuracy_results['k'], accuracy_results['MAE'], 'o-', label='MAE', linewidth=2)
    axes[0,0].plot(accuracy_results['k'], accuracy_results['RMSE'], 's-', label='RMSE', linewidth=2)
    axes[0,0].set_xlabel('Number of Similar Users (k)')
    axes[0,0].set_ylabel('Error')
    axes[0,0].set_title('Prediction Accuracy vs K')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

# 2. Quality Metrics Distribution
if not quality_results.empty:
    axes[0,1].scatter(quality_results['user_avg_rating'], quality_results['avg_predicted_rating'], alpha=0.7)
    axes[0,1].plot([5, 10], [5, 10], 'r--', alpha=0.5)  # Perfect prediction line
    axes[0,1].set_xlabel('User Average Rating')
    axes[0,1].set_ylabel('Average Predicted Rating')
    axes[0,1].set_title('Prediction Quality by User')
    axes[0,1].grid(True, alpha=0.3)

# 3. Genre Diversity
if not quality_results.empty:
    axes[1,0].bar(range(len(quality_results)), quality_results['genre_diversity'])
    axes[1,0].set_xlabel('User Index')
    axes[1,0].set_ylabel('Number of Unique Genres')
    axes[1,0].set_title('Recommendation Diversity by User')
    axes[1,0].grid(True, alpha=0.3)

# 4. Performance Comparison
if not comparison_results.empty:
    x_pos = np.arange(len(comparison_results))
    width = 0.35
    
    axes[1,1].bar(x_pos - width/2, comparison_results['cosine_time'], width, label='Cosine Similarity')
    axes[1,1].bar(x_pos + width/2, comparison_results['original_time'], width, label='Original Algorithm')
    axes[1,1].set_xlabel('User Index')
    axes[1,1].set_ylabel('Time (seconds)')
    axes[1,1].set_title('Performance Comparison')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*60)
print("📊 EVALUATION SUMMARY")
print("="*60)

if not accuracy_results.empty:
    best_k = accuracy_results.loc[accuracy_results['MAE'].idxmin()]['k']
    best_mae = accuracy_results['MAE'].min()
    print(f"🎯 Best Prediction Accuracy: MAE = {best_mae:.3f} at k = {best_k}")

if not quality_results.empty:
    avg_diversity = quality_results['genre_diversity'].mean()
    print(f"🎭 Average Genre Diversity: {avg_diversity:.1f} unique genres per user")

if not comparison_results.empty:
    speedup = comparison_results['original_time'].mean() / comparison_results['cosine_time'].mean()
    print(f"⚡ Performance: Pure cosine similarity is {speedup:.1f}x {'faster' if speedup > 1 else 'slower'} than original")
    avg_overlap = comparison_results['overlap'].mean()
    print(f"🔄 Average Recommendation Overlap: {avg_overlap:.1f}/10 movies")

print(f"\n💾 Memory Usage: {perf_results['matrix_memory_mb']:.1f} MB for user-item matrix")
print(f"⏱️  Average Recommendation Time: {perf_results['recommendation_time_per_user']:.3f}s per user")

print("\n✅ Evaluation Complete!")

## 9. Conclusions and Insights

### Key Findings:

1. **Prediction Accuracy**: 
   - Pure cosine similarity provides reasonable prediction accuracy
   - Optimal k value (number of similar users) varies based on data sparsity
   - Performance degrades with very high or very low k values

2. **Recommendation Quality**:
   - Cosine similarity tends to recommend movies similar to user preferences
   - Good diversity in genres when sufficient user data is available
   - Quality depends heavily on the number of ratings per user

3. **Performance**:
   - Pure cosine similarity is computationally efficient for similarity calculation
   - Memory usage scales with number of users and items
   - Performance bottleneck is in rating prediction for many items

4. **Comparison with Original Algorithm**:
   - Pure cosine similarity provides different but valid recommendations
   - Original hybrid approach may provide more diverse recommendations
   - Trade-off between simplicity and recommendation quality

### Recommendations for Improvement:

1. **Optimize k value**: Use cross-validation to find optimal k for your dataset
2. **Handle cold start**: Implement content-based fallback for new users
3. **Sparse data**: Consider matrix factorization for better handling of sparse ratings
4. **Performance**: Implement approximate similarity for large-scale systems
5. **Diversity**: Add diversity constraints to avoid over-specialization


In [ ]:
# Save results for further analysis
print("💾 Saving evaluation results...")

# Create results directory if it doesn't exist
import os
os.makedirs('evaluation_results', exist_ok=True)

# Save all results
if not accuracy_results.empty:
    accuracy_results.to_csv('evaluation_results/accuracy_results.csv', index=False)
    print("✅ Accuracy results saved")

if not quality_results.empty:
    quality_results.to_csv('evaluation_results/quality_results.csv', index=False)
    print("✅ Quality results saved")

if not comparison_results.empty:
    comparison_results.to_csv('evaluation_results/comparison_results.csv', index=False)
    print("✅ Comparison results saved")

# Save performance metrics
with open('evaluation_results/performance_metrics.json', 'w') as f:
    import json
    json.dump(perf_results, f, indent=2)
    print("✅ Performance metrics saved")

print("\n🎉 All evaluation results saved to 'evaluation_results' directory!")